In [47]:
#Import needed modules
import os
import sys

import ArticleImportUtils as AIU
import DisplayArticlesUtilities as DAU


from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, MBartForConditionalGeneration, MBart50TokenizerFast
from dotenv import load_dotenv
from newsapi import NewsApiClient



In [48]:
#Create variables to hold country and language codes
countryCode = ""
languageCode = ""

#Create a list of the available country codes that include ae,ar,at,au,be,bg,br,ca,ch,cn,co,cu,cz,de,eg,fr,gb,gr,hk,hu,id,ie,il,in,it,jp,kr,lt,lv,ma,mx,my,ng,nl,no,nz,ph,pl,pt,ro,rs,ru,sa,se,sg,si,sk,th,tr,tw,ua,us,ve,za
validCountryCodes = ["ae","ar","at","au","be","bg","br","ca","ch","cn","co","cu","cz","de","eg","fr","gb","gr","hk","hu","id","ie","il","in","it","jp","kr","lt","lv","ma","mx","my","ng","nl","no","nz","ph","pl","pt","ro","rs","ru","sa","se","sg","si","sk","th","tr","tw","ua","us","ve","za","None"]

#Create a list of the available language codes that include ar,de,en,es,fr,he,it,nl,no,pt,ru,se,ud,zh
validLanguageCodes = ["ar","de","en","es","fr","he","it","nl","no","pt","ru","se","ud","zh","None"]

#Ask the user to input the country code if not valid ask again
while countryCode not in validCountryCodes:
    countryCode = input("Please enter the country code you would like to search for articles in: ")
    if countryCode != "None":
        countryCode = countryCode.lower()

#Ask the user to input the language code if not valid ask again
while languageCode not in validLanguageCodes:
    languageCode = input("Please enter the language code you would like to search for articles in: ")
    if languageCode != "None":
        languageCode = languageCode.lower()



In [49]:
print("Country Code: " + countryCode)
print("Language Code: " + languageCode)


#Get News Headlines
if countryCode == "None" and languageCode == "None":
    top_headlines = AIU.get_top_headlines()
elif countryCode == "None":
    top_headlines = AIU.get_top_headlines(language=languageCode)
elif languageCode == "None":
    top_headlines = AIU.get_top_headlines(country=countryCode)
else:
    top_headlines = AIU.get_top_headlines(country=countryCode, language=languageCode)

articles = DAU.CreateArticleList(top_headlines)

print(articles)

#Pass Article Dictionary to DisplayArticlesUtils
DAU.DisplayArticleInfo(articles)

Country Code: fr
Language Code: fr


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'title': "En direct, l'attaque iranienne contre Israël : le Conseil de sécurité de l'ONU réuni en urgence dimanche, des drones et des missiles interceptés - Le Monde", 'author': 'Le Monde', 'description': None, 'content': None, 'url': 'https://news.google.com/rss/articles/CBMiyQFodHRwczovL3d3dy5sZW1vbmRlLmZyL2ludGVybmF0aW9uYWwvbGl2ZS8yMDI0LzA0LzEzL2VuLWRpcmVjdC1sLWlyYW4tYS1sYW5jZS11bmUtYXR0YXF1ZS1kZS1kcm9uZXMtY29udHJlLWlzcmFlbC1sZXMtZXRhdHMtdW5pcy1yZWFmZmlybWVudC1sZXVyLXNvdXRpZW4tYS1sYS1kZWZlbnNlLWRlLWwtZXRhdC1oZWJyZXVfNjIyNzY3Ml8zMjEwLmh0bWzSAQA?oc=5', 'source': 'Google News', 'published_date': '2024-04-14T04:12:20Z', 'sentiment': {'label': 'POSITIVE', 'score': 0.923418402671814}, 'keywords': ['en', 'direct', ',', 'l', "'", 'at', '##ta', '##que', 'iran', '##ien', '##ne', 'con', '##tre', 'israel', ':', 'le', 'con', '##sei', '##l', 'de', 'sec', '##uri', '##te', 'de', 'l', "'", 'on', '##u', 're', '##uni', 'en', 'urge', '##nce', 'dim', '##an', '##che', ',', 'des', 'drones', 'et', 'des',

In [50]:
#Take the first article and translate it to English if it is not in English
article = articles[0]
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer.src_lang = languageCode + "_" + languageCode.upper()



if languageCode != "en":
    print("Translating article to English...")

    model = MBartForConditionalGeneration.from_pretrained(model_name)

    # Translate the article title to English
    input_text = article['title']
    encoded_text = tokenizer(input_text, return_tensors="pt")

    output_ids = model.generate(**encoded_text, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])

    # Retrieve the text from the special characters.
    translated_title = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    print("Translated Title: " + translated_title)

    print("Original Title: " + article['title'])

else:
    print("Article is already in English")

Translating article to English...
Translated Title: Iranian attack on Israel: U.N. Security Council emergency meeting Sunday, drones and missiles intercepted - Le Monde
Original Title: En direct, l'attaque iranienne contre Israël : le Conseil de sécurité de l'ONU réuni en urgence dimanche, des drones et des missiles interceptés - Le Monde
